# core

> Fill in a module description here

In [ ]:
#| default_exp core

In [ ]:
#| hide
import os
import openai
from fastcore.all import AttrDict, maybe_attr
from IPython.core.magic import register_cell_magic
openai.api_key = os.getenv("OPENAI_API_KEY")

In [ ]:
context = """
These tables are stored in Google BigQuery, All of the tables are listed below, along with a json extract of the first two rows.  The pattern is:

Query
----
Results

SELECT * FROM `bigquery-public-data.github_repos.commits` LIMIT 2
----
[{
  "commit": "aa358905a1b12c6fa43b6e877e907fc9d36ff0b9",
  "tree": "df3f8bf61bf1cb0dff3a86ebe18671792c2d4f27",
  "parent": ["ea230a45a0e97e4d95b5f7fae9ce7ef659b60291"],
  "author": {
    "name": "conda-forge-coordinator",
    "email": "e5ec32c8593b6ea3202de4b38c1a852e362e575b@gmail.com",
    "time_sec": "1647357073",
    "tz_offset": "0",
    "date": {
      "seconds": "1647357073",
      "nanos": null
    }
  },
  "committer": {
    "name": "conda-forge-coordinator",
    "email": "e5ec32c8593b6ea3202de4b38c1a852e362e575b@gmail.com",
    "time_sec": "1647357073",
    "tz_offset": "0",
    "date": {
      "seconds": "1647357073",
      "nanos": null
    }
  },
  "subject": "Updated the qcengine feedstock.",
  "message": "Updated the qcengine feedstock.",
  "trailer": [],
  "difference": [],
  "difference_truncated": "true",
  "repo_name": ["conda-forge/feedstocks"],
  "encoding": null
}, {
  "commit": "5a6b6d6d29489f8587b247313804d70c45d0981f",
  "tree": "ff89accb7e283ca88027d790cdb8bacf373895ab",
  "parent": ["4ee369feb64ee97d71da732da0e78477efbadd76", "6aae5a3059865fa85940e1af13157920ba2efb5e"],
  "author": {
    "name": "Rob Allen",
    "email": "7e09c9d3e96378bf549fc283fd6e1e5b7014cc33@akrabat.com",
    "time_sec": "1323443117",
    "tz_offset": "0",
    "date": {
      "seconds": "1323443117",
      "nanos": null
    }
  },
  "committer": {
    "name": "Rob Allen",
    "email": "7e09c9d3e96378bf549fc283fd6e1e5b7014cc33@akrabat.com",
    "time_sec": "1323443117",
    "tz_offset": "0",
    "date": {
      "seconds": "1323443117",
      "nanos": null
    }
  },
  "subject": "Merge remote-tracking branch \u0027weierophinney/hotfix/amf-authentication\u0027",
  "message": "Merge remote-tracking branch \u0027weierophinney/hotfix/amf-authentication\u0027\n",
  "trailer": [],
  "difference": [],
  "difference_truncated": null,
  "repo_name": ["MadCat34/zend-escaper", "Maks3w/zend-escaper", "zendframework/zend-escaper"],
  "encoding": null
}]

SELECT * FROM `bigquery-public-data.github_repos.contents` LIMIT 2
----
[{
  "id": "ee4a3a38b49a6e43cef4c734b45f698a350d6ba0",
  "size": "40365",
  "content": null,
  "binary": "true",
  "copies": "1"
}, {
  "id": "d84baedb35d259fb6a165bc9e989367bf5add523",
  "size": "84150",
  "content": null,
  "binary": "true",
  "copies": "1"
}]

SELECT * FROM `bigquery-public-data.github_repos.files` LIMIT 2
----
[{
  "repo_name": "nicolasbracigliano/NB-Site",
  "ref": "refs/heads/gh-pages",
  "path": "node_modules/fsevents/node_modules/.bin/rimraf",
  "mode": "40960",
  "id": "4cd49a49ddfc17a0c038a9c11d3abbb8181dc025",
  "symlink_target": "../rimraf/bin.js"
}, {
  "repo_name": "nicolasbracigliano/NB-Site",
  "ref": "refs/heads/gh-pages",
  "path": "node_modules/fsevents/node_modules/.bin/sshpk-conv",
  "mode": "40960",
  "id": "a2a295c8014c8c4fb4d0a35adf4567bd52cd82e4",
  "symlink_target": "../sshpk/bin/sshpk-conv"
}]

SELECT * FROM `bigquery-public-data.github_repos.languages` LIMIT 2
----
[{
  "repo_name": "lemi136/puntovent",
  "language": [{
    "name": "C",
    "bytes": "80"
  }]
}, {
  "repo_name": "taxigps/nctool",
  "language": [{
    "name": "C",
    "bytes": "4461"
  }]
}]

SELECT * FROM `bigquery-public-data.github_repos.licenses` LIMIT 2
----
[{
  "repo_name": "autarch/Dist-Zilla-Plugin-Test-TidyAll",
  "license": "artistic-2.0"
}, {
  "repo_name": "thundergnat/Prime-Factor",
  "license": "artistic-2.0"
}]

Given these tables and schema, please construct a SQL query in BigQuery that answers the following question:

"""

In [ ]:
#| export
def chat(query):
    completion = AttrDict(openai.ChatCompletion.create(
      model="gpt-3.5-turbo",
      messages=[
        {"role": "user", 
         "content": f"{context} {query}"}]))
    if hasattr(completion, 'choices'):
        return completion.choices[0].message.content.strip()
    else: None


In [ ]:
chat("How are you feeling?")

'As an AI language model, I do not have feelings. Is there a specific question about the GitHub repositories dataset that you would like me to help you with?'

In [ ]:
#|export
@register_cell_magic
def bq(line, cell):
    "Ask BigQuery A Question"
    question = cell.strip()
    print(chat(question))

In [ ]:
%%bq

What is the most popular kind of license on GitHub?

SELECT license, COUNT(*) AS count
FROM `bigquery-public-data.github_repos.licenses`
GROUP BY license
ORDER BY count DESC
LIMIT 1


In [ ]:
%%bq

Who are the top 10 most committers on GitHub?

SELECT author.name AS committer, COUNT(*) AS commits_count
FROM `bigquery-public-data.github_repos.commits`
GROUP BY committer
ORDER BY commits_count DESC
LIMIT 10


In [ ]:
%%bq 

How many python files are there on GitHub in mit licensed repos?

SELECT COUNT(*) as num_python_files
FROM `bigquery-public-data.github_repos.files`
WHERE RIGHT(path, 3) = '.py'
AND repo_name IN (
  SELECT repo_name
  FROM `bigquery-public-data.github_repos.licenses`
  WHERE license = 'mit'
)
